# Imports

In [ ]:
import numpy as np
import numpy.linalg as linalg
from time import time
from simulationTools import *

# Generic Example

### Tweakable Parameters

In [ ]:
# Number of cores for computing
nCores = 2

# Number of spins in chain
n = 8

# Define Hamiltonian args=(nSpins,h,g,J)
H = transverse_field_ising_hamiltonian(n, .5, 1.05, 1)
# Power Law Ising Hamiltonian args=(nSpins,h,g,J,lmax,zeta)
#hz = [.375, -.375, .375, -.375, .375, -.375, .375, -.375]
#H = power_law_ising_hamiltonian(n, hz, 1.05, 1, 5, 6)

# Define initial density matrix.
# For zero temperature use beta=np.inf. For infinite temperature, use beta=0.
beta = 1
Lambda, Q = linalg.eigh(H.todense())
rho = thermal_density_matrix(Lambda, Q, beta)

# Define weak measurement projection operators
# args = (nSpins, nMeas, sign, axis)
# nSpins - length of spin chain (set at top)
# nMeas - site to measure at (labelled 0,1,..,n-1) (usually use 0)
# sign - +1 for project onto aligned spins, -1 for anti-aligned
# axis - spherical polar coordinates for measurement axis [theta, phi]
vAxis = [0,0]
V1 = construct_local_projector(n, 0, 0, vAxis)
V2 = construct_local_projector(n, 0, 1, vAxis)

# Define strong measurement projection operators
# Same syntax as weak measurement operators.
# Usually choose to measure at the last spin site (n-1)
# Need to be complete, so should have same parameters but
# opposite signs
wAxis = [np.pi/2,0]
W1 = construct_local_projector(n, n-1, 0, wAxis)
W2 = construct_local_projector(n, n-1, 1, wAxis)
Wlist = [W1, W2]

# Couplings that appear in weak measurement Kraus operators
# args=(x0, Delta, gtilde, xValues, hbar)
# p0 - central value of momentum wavepacket for readout particle
# Delta - uncertainty in momentum wavepacket
# gtilde - weak coupling between system and detector
# xValues - possible readout positions of particle
# WARNINGS: 1) The list of xValues cannot be too large. The maximization
#              step requires holding the square of the number of entries 
#              in memory. More than a few thousand entries is likely to 
#              crash your laptop.
#           2) Small values of Delta (<.1) create significant spread in
#              position space. Check that the bulk of the probability is
#              in your list of xValues using the plots below.
xValues = np.arange(-30,30,.1)
pg1 = weak_measurement_couplings(10, .1, .02, xValues)
pg2 = weak_measurement_couplings(10, .1, .02, xValues)

# Time values to plot inequality at
tList = np.linspace(0,10,40)

# Smoothing parameter for smoothed inequality
eps = .01

### Visualize Weak Measurement Couplings

In [ ]:
visualize_weak_couplings(pg1, xValues)

### Run Simulation

In [ ]:
t1 = time()

LHSneu, LHSmm, LHSneuTerms, LHSmmTerms, rhoF, rhoR, f, fOrd, fTerm, qProbs = simulation(V1, V2, pg1, pg2, Wlist, H, rho, tList, nWorks=nCores)

t2 = time()
print("Runtime = " + repr(t2-t1) + " seconds")

### Visualize Inequality

In [ ]:
visualize_inequality(tList, LHSneu, LHSmm, f, fOrd, fTerm, qProbs)

### Visualize Smoothed Inequality

In [ ]:
Hmin, Hmax, Hmineps, Hmaxeps = visualize_smoothed_inequality(tList, rhoF, rhoR, f, eps)

### Visualize Density Matrices of Detector

In [ ]:
visualize_detector_state(xValues, rhoF, 'Forward Detector')
visualize_detector_state(xValues, rhoR, 'Reverse Detector')

### Visualize All Quasiprobabilities

In [ ]:
tA1213, tA2223, tA1223 = create_all_quasiprobs(V1, V2, pg1, pg2, Wlist, H, tList, hbar=1)
visualize_all_quasiprobs(tList, tA1213, tA2223, tA1223)

# Examples

## 0) Integrable Longitudinal Ising, Coarse Grained, Weak Coupling

In [ ]:
t1 = time() # Expected run-time: 2 minutes

nCores = 2
n = 8
H = transverse_field_ising_hamiltonian(n, 0, 1.05, 1)
beta = 1
Lambda, Q = linalg.eigh(H.todense())
rho = thermal_density_matrix(Lambda, Q, beta)
vAxis = [0,0]
V1 = construct_local_projector(n, 0, 0, vAxis)
V2 = construct_local_projector(n, 0, 1, vAxis)
xValues = np.arange(-30,30,.1)
pg1 = weak_measurement_couplings(10, .1, .02, xValues)
pg2 = weak_measurement_couplings(10, .1, .02, xValues)
wAxis = [0,0]
W1 = construct_local_projector(n, n-1, 0, wAxis)
W2 = construct_local_projector(n, n-1, 1, wAxis)
Wlist = [W1, W2]
tList = np.linspace(0,20,101)
eps=.01

visualize_weak_couplings(pg1, xValues)

LHSneu, LHSmm, LHSneuTerms, LHSmmTerms, rhoF, rhoR, f, fOrd, fTerm, qProbs = simulation(V1, V2, pg1, pg2, Wlist, H, rho, tList, nWorks=nCores)

visualize_inequality(tList, LHSneu, LHSmm, f, fOrd, fTerm, qProbs)

visualize_smoothed_inequality(tList, rhoF, rhoR, f, eps)

visualize_detector_state(xValues, rhoF, 'Forward Detector')
visualize_detector_state(xValues, rhoR, 'Reverse Detector')

tA1213, tA2223, tA1223 = create_all_quasiprobs(V1, V2, pg1, pg2, Wlist, H, tList, hbar=1)
visualize_all_quasiprobs(tList, tA1213, tA2223, tA1223)

t2 = time()
print("Runtime = " + repr(t2-t1) + " seconds")

## 1) Longitudinal Ising, Coarse Grained, Weak Coupling

In [ ]:
t1 = time() # Expected run-time: 2 minutes

nCores = 2
n = 8
H = transverse_field_ising_hamiltonian(n, .5, 1.05, 1)
beta = 1
Lambda, Q = linalg.eigh(H.todense())
rho = thermal_density_matrix(Lambda, Q, beta)
vAxis = [0,0]
V1 = construct_local_projector(n, 0, 0, vAxis)
V2 = construct_local_projector(n, 0, 1, vAxis)
xValues = np.arange(-30,30,.1)
pg1 = weak_measurement_couplings(10, .1, .02, xValues)
pg2 = weak_measurement_couplings(10, .1, .02, xValues)
wAxis = [0,0]
W1 = construct_local_projector(n, n-1, 0, wAxis)
W2 = construct_local_projector(n, n-1, 1, wAxis)
Wlist = [W1, W2]
tList = np.linspace(0,10,101)
eps=.01

visualize_weak_couplings(pg1, xValues)

LHSneu, LHSmm, LHSneuTerms, LHSmmTerms, rhoF, rhoR, f, fOrd, fTerm, qProbs = simulation(V1, V2, pg1, pg2, Wlist, H, rho, tList, nWorks=nCores)

visualize_inequality(tList, LHSneu, LHSmm, f, fOrd, fTerm, qProbs)

visualize_smoothed_inequality(tList, rhoF, rhoR, f, eps)

visualize_detector_state(xValues, rhoF, 'Forward Detector')
visualize_detector_state(xValues, rhoR, 'Reverse Detector')

tA1213, tA2223, tA1223 = create_all_quasiprobs(V1, V2, pg1, pg2, Wlist, H, tList, hbar=1)
visualize_all_quasiprobs(tList, tA1213, tA2223, tA1223)

t2 = time()
print("Runtime = " + repr(t2-t1) + " seconds")

## 2) Longitudinal Ising, Coarse Grained, Strong Coupling

In [ ]:
t1 = time() # Expected run-time: 3 minutes

nCores = 2
n = 8
H = transverse_field_ising_hamiltonian(n, .5, 1.05, 1)
beta = 1
Lambda, Q = linalg.eigh(H.todense())
rho = thermal_density_matrix(Lambda, Q, beta)
vAxis = [0,0]
V1 = construct_local_projector(n, 0, 0, vAxis)
V2 = construct_local_projector(n, 0, 0, vAxis)
xValues = np.arange(-30,30,.1)
pg1 = weak_measurement_couplings(10, .1, .16, xValues)
pg2 = weak_measurement_couplings(10, .1, .16, xValues)
wAxis = [0,0]
W1 = construct_local_projector(n, n-1, 0, wAxis)
W2 = construct_local_projector(n, n-1, 1, wAxis)
Wlist = [W1, W2]
tList = np.linspace(0,10,101)
eps=.01

visualize_weak_couplings(pg1, xValues)

LHSneu, LHSmm, LHSneuTerms, LHSmmTerms, rhoF, rhoR, f, aOrd, aTerm, qProbs = simulation_strong(V1, V2, pg1, pg2, Wlist, H, rho, tList, nWorks=nCores)

visualize_inequality_strong(tList, LHSneu, LHSmm, f, aOrd, aTerm, qProbs)

visualize_smoothed_inequality(tList, rhoF, rhoR, f, eps)

visualize_detector_state(xValues, rhoF, 'Forward Detector')
visualize_detector_state(xValues, rhoR, 'Reverse Detector')

t2 = time()
print("Runtime = " + repr(t2-t1) + " seconds")

## 3) Longitudinal Ising, Partially Fine Grained, Weak Coupling

In [ ]:
t1 = time() # Expected run-time: 20 minutes

nCores = 2
n = 8
H = transverse_field_ising_hamiltonian(n, .5, 1.05, 1)
beta = 1
Lambda, Q = linalg.eigh(H.todense())
rho = thermal_density_matrix(Lambda, Q, beta)
vAxis = [0,0]
V1 = construct_local_projector(n, 0, 0, vAxis)
V2 = construct_local_projector(n, 0, 1, vAxis)
xValues = np.arange(-30,30,.1)
pg1 = weak_measurement_couplings(10, .1, .02, xValues)
pg2 = weak_measurement_couplings(10, .1, .02, xValues)
wAxisList = [None, None, None, None, [0,0], [0,0], [0,0], [0,0]]
Wlist = construct_complete_projector_list(n, wAxisList)
tList = np.linspace(0,10,101)
eps = .01

visualize_weak_couplings(pg1, xValues)

LHSneu, LHSmm, LHSneuTerms, LHSmmTerms, rhoF, rhoR, f, fOrd, fTerm, qProbs = simulation(V1, V2, pg1, pg2, Wlist, H, rho, tList, nWorks=nCores)

visualize_inequality(tList, LHSneu, LHSmm, f, fOrd, fTerm, qProbs)

visualize_smoothed_inequality(tList, rhoF, rhoR, f, eps)

visualize_detector_state(xValues, rhoF, 'Forward Detector')
visualize_detector_state(xValues, rhoR, 'Reverse Detector')

tA1213, tA2223, tA1223 = create_all_quasiprobs(V1, V2, pg1, pg2, Wlist, H, tList, hbar=1)
visualize_all_quasiprobs(tList, tA1213, tA2223, tA1223)

t2 = time()
print("Runtime = " + repr(t2-t1) + " seconds")

## 4) Longitudinal Ising, Fine Grained, Weak Coupling

In [ ]:
t1 = time() # Expected run-time: 3 hours

nCores = 2
n = 8
H = transverse_field_ising_hamiltonian(n, .5, 1.05, 1)
beta = 1
Lambda, Q = linalg.eigh(H.todense())
rho = thermal_density_matrix(Lambda, Q, beta)
vAxis = [0,0]
V1 = construct_local_projector(n, 0, 0, vAxis)
V2 = construct_local_projector(n, 0, 1, vAxis)
xValues = np.arange(-30,30,.1)
pg1 = weak_measurement_couplings(10, .1, .02, xValues)
pg2 = weak_measurement_couplings(10, .1, .02, xValues)
wAxisList = [[0,0], ]*n
Wlist = construct_complete_projector_list(n, wAxisList)
tList = np.linspace(0,10,101)
eps = .01

visualize_weak_couplings(pg1, xValues)

LHSneu, LHSmm, LHSneuTerms, LHSmmTerms, rhoF, rhoR, f, fOrd, fTerm, qProbs = simulation(V1, V2, pg1, pg2, Wlist, H, rho, tList, nWorks=nCores)

visualize_inequality(tList, LHSneu, LHSmm, f, fOrd, fTerm, qProbs)

visualize_smoothed_inequality(tList, rhoF, rhoR, f, eps)

tA1213, tA2223, tA1223 = create_all_quasiprobs(V1, V2, pg1, pg2, Wlist, H, tList, hbar=1)
visualize_all_quasiprobs(tList, tA1213, tA2223, tA1223)

t2 = time()
print("Runtime = " + repr(t2-t1) + " seconds")

## 5) Power-Law Ising, Coarse Grained, Weak Coupling

In [ ]:
t1 = time() # Expected run-time: 2 minutes

nCores = 2
n = 8
hz = [.375, -.375, .375, -.375, .375, -.375, .375, -.375]
H = power_law_ising_hamiltonian(n, hz, 1.05, 1, 5, 6)
beta = 1
Lambda, Q = linalg.eigh(H.todense())
rho = thermal_density_matrix(Lambda, Q, beta)
vAxis = [0,0]
V1 = construct_local_projector(n, 0, 0, vAxis)
V2 = construct_local_projector(n, 0, 1, vAxis)
xValues = np.arange(-30,30,.1)
pg1 = weak_measurement_couplings(10, .1, .02, xValues)
pg2 = weak_measurement_couplings(10, .1, .02, xValues)
wAxis = [0,0]
W1 = construct_local_projector(n, n-1, 0, wAxis)
W2 = construct_local_projector(n, n-1, 1, wAxis)
Wlist = [W1, W2]
tList = np.linspace(0,10,101)
eps=.01

visualize_weak_couplings(pg1, xValues)

LHSneu, LHSmm, LHSneuTerms, LHSmmTerms, rhoF, rhoR, f, fOrd, fTerm, qProbs = simulation(V1, V2, pg1, pg2, Wlist, H, rho, tList, nWorks=nCores)

visualize_inequality(tList, LHSneu, LHSmm, f, fOrd, fTerm, qProbs)

visualize_smoothed_inequality(tList, rhoF, rhoR, f, eps)

visualize_detector_state(xValues, rhoF, 'Forward Detector')
visualize_detector_state(xValues, rhoR, 'Reverse Detector')

tA1213, tA2223, tA1223 = create_all_quasiprobs(V1, V2, pg1, pg2, Wlist, H, tList, hbar=1)
visualize_all_quasiprobs(tList, tA1213, tA2223, tA1223)

t2 = time()
print("Runtime = " + repr(t2-t1) + " seconds")

## 6) Power-Law Ising, Coarse Grained, Strong Coupling

In [ ]:
t1 = time() # Expected run-time: 2 minutes

nCores = 2
n = 8
hz = [.375, -.375, .375, -.375, .375, -.375, .375, -.375]
H = power_law_ising_hamiltonian(n, hz, 1.05, 1, 5, 6)
beta = 1
Lambda, Q = linalg.eigh(H.todense())
rho = thermal_density_matrix(Lambda, Q, beta)
vAxis = [0,0]
V1 = construct_local_projector(n, 0, 0, vAxis)
V2 = construct_local_projector(n, 0, 1, vAxis)
xValues = np.arange(-30,30,.1)
pg1 = weak_measurement_couplings(10, .1, .16, xValues)
pg2 = weak_measurement_couplings(10, .1, .16, xValues)
wAxis = [0,0]
W1 = construct_local_projector(n, n-1, 0, wAxis)
W2 = construct_local_projector(n, n-1, 1, wAxis)
Wlist = [W1, W2]
tList = np.linspace(0,10,101)
eps=.01

visualize_weak_couplings(pg1, xValues)

LHSneu, LHSmm, LHSneuTerms, LHSmmTerms, rhoF, rhoR, f, aOrd, aTerm, qProbs = simulation_strong(V1, V2, pg1, pg2, Wlist, H, rho, tList, nWorks=nCores)

visualize_inequality_strong(tList, LHSneu, LHSmm, f, aOrd, aTerm, qProbs)

visualize_smoothed_inequality(tList, rhoF, rhoR, f, eps)

visualize_detector_state(xValues, rhoF, 'Forward Detector')
visualize_detector_state(xValues, rhoR, 'Reverse Detector')

t2 = time()
print("Runtime = " + repr(t2-t1) + " seconds")

## 7) Power-Law Ising, Coarse Grained, Strong Coupling, Engineered Initial State

In [ ]:
t1 = time() # Expected run-time: 3 minutes

nCores = 2
n = 8
hz = [.375, -.375, .375, -.375, .375, -.375, .375, -.375]
H = power_law_ising_hamiltonian(n, hz, 1.05, 1, 5, 6)
vAxis = [0,0]
V1 = construct_local_projector(n, 0, 0, vAxis)
V2 = construct_local_projector(n, 0, 1, vAxis)
xValues = np.arange(-30,30,.1)
pg1 = weak_measurement_couplings(10, .1, .16, xValues)
pg2 = weak_measurement_couplings(10, .1, .16, xValues)
wAxis = [0,0]
W1 = construct_local_projector(n, n-1, 0, wAxis)
W2 = construct_local_projector(n, n-1, 1, wAxis)
Wlist = [W1, W2]
tList = np.linspace(0,10,101)
eps=.01

t_unscramble = 5
Lambda, Q = linalg.eigh(H.todense())
rho_unscrambled = Wlist[0]/np.trace(Wlist[0].todense())
U_scramble = get_U(Lambda, Q, t_unscramble)
rho = U_scramble.getH() @ rho_unscrambled @ U_scramble

visualize_weak_couplings(pg1, xValues)

LHSneu, LHSmm, LHSneuTerms, LHSmmTerms, rhoF, rhoR, f, aOrd, aTerm, qProbs = simulation_strong(V1, V2, pg1, pg2, Wlist, H, rho, tList, nWorks=nCores)

visualize_inequality_strong(tList, LHSneu, LHSmm, f, aOrd, aTerm, qProbs)

visualize_smoothed_inequality(tList, rhoF, rhoR, f, eps)

visualize_detector_state(xValues, rhoF, 'Forward Detector')
visualize_detector_state(xValues, rhoR, 'Reverse Detector')

t2 = time()
print("Runtime = " + repr(t2-t1) + " seconds")

## 8) Power-Law Ising, Partially Fine Grained, Weak Coupling

In [ ]:
t1 = time() # Expected run-time: 20 minutes

nCores = 2
n = 8
hz = [.375, -.375, .375, -.375, .375, -.375, .375, -.375]
H = power_law_ising_hamiltonian(n, hz, 1.05, 1, 5, 6)
beta = 1
Lambda, Q = linalg.eigh(H.todense())
rho = thermal_density_matrix(Lambda, Q, beta)
vAxis = [0,0]
V1 = construct_local_projector(n, 0, 0, vAxis)
V2 = construct_local_projector(n, 0, 1, vAxis)
xValues = np.arange(-30,30,.1)
pg1 = weak_measurement_couplings(10, .1, .02, xValues)
pg2 = weak_measurement_couplings(10, .1, .02, xValues)
wAxisList = [None, None, None, None, [0,0], [0,0], [0,0], [0,0]]
Wlist = construct_complete_projector_list(n, wAxisList)
tList = np.linspace(0,10,101)
eps = .01

visualize_weak_couplings(pg1, xValues)

LHSneu, LHSmm, LHSneuTerms, LHSmmTerms, rhoF, rhoR, f, fOrd, fTerm, qProbs = simulation(V1, V2, pg1, pg2, Wlist, H, rho, tList, nWorks=nCores)

visualize_inequality(tList, LHSneu, LHSmm, f, fOrd, fTerm, qProbs)

visualize_smoothed_inequality(tList, rhoF, rhoR, f, eps)

visualize_detector_state(xValues, rhoF, 'Forward Detector')
visualize_detector_state(xValues, rhoR, 'Reverse Detector')

tA1213, tA2223, tA1223 = create_all_quasiprobs(V1, V2, pg1, pg2, Wlist, H, tList, hbar=1)
visualize_all_quasiprobs(tList, tA1213, tA2223, tA1223)

t2 = time()
print("Runtime = " + repr(t2-t1) + " seconds")

## 9) Power-Law Ising, Fine Grained, Weak Coupling

In [ ]:
t1 = time() # Expected run-time: 3 hours

nCores = 2
n = 8
hz = [.375, -.375, .375, -.375, .375, -.375, .375, -.375]
H = power_law_ising_hamiltonian(n, hz, 1.05, 1, 5, 6)
beta = 1
Lambda, Q = linalg.eigh(H.todense())
rho = thermal_density_matrix(Lambda, Q, beta)
vAxis = [0,0]
V1 = construct_local_projector(n, 0, 0, vAxis)
V2 = construct_local_projector(n, 0, 1, vAxis)
xValues = np.arange(-30,30,.1)
pg1 = weak_measurement_couplings(10, .1, .02, xValues)
pg2 = weak_measurement_couplings(10, .1, .02, xValues)
wAxisList = [[0,0], ]*n
Wlist = construct_complete_projector_list(n, wAxisList)
tList = np.linspace(0,10,101)
eps = .01

visualize_weak_couplings(pg1, xValues)

LHSneu, LHSmm, LHSneuTerms, LHSmmTerms, rhoF, rhoR, f, fOrd, fTerm, qProbs = simulation(V1, V2, pg1, pg2, Wlist, H, rho, tList, nWorks=nCores)

visualize_inequality(tList, LHSneu, LHSmm, f, fOrd, fTerm, qProbs)

visualize_smoothed_inequality(tList, rhoF, rhoR, f, eps)

tA1213, tA2223, tA1223 = create_all_quasiprobs(V1, V2, pg1, pg2, Wlist, H, tList, hbar=1)
visualize_all_quasiprobs(tList, tA1213, tA2223, tA1223)

t2 = time()
print("Runtime = " + repr(t2-t1) + " seconds")

## 10) Power-Law Ising, Fine Grained, Weak Coupling, Engineered Initial State

In [ ]:
t1 = time() # Expected run-time: 3 hours

nCores = 2
n = 8
hz = [.375, -.375, .375, -.375, .375, -.375, .375, -.375]
H = power_law_ising_hamiltonian(n, hz, 1.05, 1, 5, 6)

vAxis = [0,0]
V1 = construct_local_projector(n, 0, 0, vAxis)
V2 = construct_local_projector(n, 0, 1, vAxis)
xValues = np.arange(-30,30,.1)
pg1 = weak_measurement_couplings(10, .1, .02, xValues)
pg2 = weak_measurement_couplings(10, .1, .02, xValues)
wAxisList = [[0,0], ]*n
Wlist = construct_complete_projector_list(n, wAxisList)
tList = np.linspace(0,10,101)
eps = .01

t_unscramble = 5
Lambda, Q = linalg.eigh(H.todense())
rho_unscrambled = Wlist[0]
U_scramble = get_U(Lambda, Q, t_unscramble)
rho = U_scramble.getH() @ rho_unscrambled @ U_scramble

visualize_weak_couplings(pg1, xValues)

LHSneu, LHSmm, LHSneuTerms, LHSmmTerms, rhoF, rhoR, f, fOrd, fTerm, qProbs = simulation(V1, V2, pg1, pg2, Wlist, H, rho, tList, nWorks=nCores)

visualize_inequality(tList, LHSneu, LHSmm, f, fOrd, fTerm, qProbs)

visualize_smoothed_inequality(tList, rhoF, rhoR, f, eps)

tA1213, tA2223, tA1223 = create_all_quasiprobs(V1, V2, pg1, pg2, Wlist, H, tList, hbar=1)
visualize_all_quasiprobs(tList, tA1213, tA2223, tA1223)

t2 = time()
print("Runtime = " + repr(t2-t1) + " seconds")